In [8]:
#trying out medical embeddings
from gensim.models import KeyedVectors
filename = 'wikipedia-pubmed-and-PMC-w2v.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [14]:
import os
import io
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

osPath = os.path.abspath("Tokenizer.ipynb")
path = 'concept_assertion_relation_training_data/beth/txt/'
files = os.listdir(path)

In [17]:
stop_words = set(stopwords.words('english'))

goods = []
bads = []
for file in files:
    #filter
    if not 'record-' in file:
        continue
    fpath = path+file
    with open(fpath) as iFile:
        for line in iFile:
            words = line.split()
            for w in words: 
                if not w in stop_words: 
                    if w in model:
                        goods.append(w)
                    else:
                        bads.append(w)

In [22]:
len(goods),len(bads),len(goods)/len(bads)

(67593, 2392, 28.25794314381271)

In [9]:
import os
tpath='concept_assertion_relation_training_data/beth/txt/'
cpath='concept_assertion_relation_training_data/beth/concept/'

In [24]:
tlist = [ f.split('.')[0] for f in os.listdir(tpath) if len(f.split('.')[0])>0 ]

In [106]:
#init
allrec = {}
for fname in tlist:
    f=open(tpath+fname+'.txt')
    rec = []
    for l in f:
        line = []
        for w in l.split():
            line.append([w.lower(),'o'])
        rec.append(line)
    allrec[fname] = rec

In [107]:
import re
for fname in tlist:
    f = open(cpath+fname+'.con')
    for i,l in enumerate(f):
        p = l.split('||')[0].split('"')[-1].strip()
        no = int(p.split()[0].split(':')[0])-1
        s = int(p.split()[0].split(':')[1])
        e = int(p.split()[1].split(':')[1])
        try:    
            allrec[fname][no][s][1] = 'b'
            if s==e:
                continue
            for j in (s+1,e):
                allrec[fname][no][j][1] = 'i'
        except IndexError:
            print(fname,no,allrec[fname][no],s,e)
            break

In [114]:
for i,line in enumerate(allrec['record-176']):
    print(i+1,line,'\n')

1 [['admission', 'o'], ['date', 'o'], [':', 'o']] 

2 [['2016-08-08', 'o']] 

3 [['discharge', 'o'], ['date', 'o'], [':', 'o']] 

4 [['2016-08-15', 'o']] 

5 [['date', 'o'], ['of', 'o'], ['birth', 'o'], [':', 'o']] 

6 [['1978-08-02', 'o']] 

7 [['sex', 'o'], [':', 'o']] 

8 [['f', 'o']] 

9 [['service:', 'o']] 

10 [['procedure', 'o'], [':', 'o']] 

11 [['cadaveric', 'b'], ['pancreas', 'i'], ['transplantation', 'i'], ['.', 'o']] 

12 [['history', 'o'], ['of', 'o'], ['present', 'o'], ['illness', 'o'], [':', 'o']] 

13 [['the', 'o'], ['patient', 'o'], ['is', 'o'], ['a', 'o'], ['37', 'o'], ['year', 'o'], ['old', 'o'], ['lady', 'o'], ['with', 'o'], ['type', 'b'], ['1', 'i'], ['diabetes', 'o'], ['mellitus', 'i'], [',', 'o'], ['who', 'o'], ['is', 'o'], ['four', 'o'], ['months', 'o'], ['postcadaveric', 'b'], ['kidney', 'i'], ['transplantation', 'i'], ['and', 'o'], ['now', 'o'], ['has', 'o'], ['good', 'o'], ['graft', 'o'], ['function', 'o'], ['.', 'o']] 

14 [['she', 'o'], ['presents', 'o'], 